# Visualize the labelled Images

You can use this notebook to visualise the images we annotated with the LabelMe tool (https://github.com/wkentaro/labelme).

Author(s): Peer Schütt

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import json
import os

from labelme import utils as labelme_utils
from utils import labelme_plot_with_custom_cmap
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# change the path to the file you want to visualize
labelme_file = "../sample_images/20250213_135748_207136_NIR_21p6575.json"

In [ ]:
evaluate_keys = ['ErdeXDuenger_1p1_100', 'ErdeXDuenger_1p1_50', 'ErdeXDuenger_1p4_100', 'ErdeXDuenger_1p4_50', 'Erde_100', 'Erde_50', 'Sand_20', 'Sand_50', 'Spectralon_x', 'Waschpulver_25', 'Waschpulver_50', "Ethanol"]
label_name_to_value = dict(zip(evaluate_keys, np.arange(1, len(evaluate_keys)+1)))

data = json.load(open(labelme_file))
input_img = labelme_utils.img_b64_to_arr(data.get("imageData"))

label_mask = np.zeros(input_img.shape, dtype=np.int32)
# Fill in the labeled regions
for i, shape in enumerate(data['shapes']):
    
    label = shape["label"]    
    points = shape["points"]
    # Convert to numpy array
    points = np.array(points, dtype=np.int32)
    
    # Create binary mask for this shape (1 for shape, 0 elsewhere)
    shape_mask = labelme_utils.shape_to_mask(
        input_img.shape, points, shape_type=shape['shape_type']
    )
    mask_value = 0
    try: 
        mask_value = label_name_to_value[label]
    except:
        mask_value = 0
        print("No entry found for ", label)
    
    if label != "road":
        # Set class index (add 1 to leave 0 as background)
        label_mask[shape_mask] = label_name_to_value[label]   

value_to_label_name = {v: k for k, v in label_name_to_value.items()}

keys_present = list(set(evaluate_keys) & set(label_name_to_value.keys()))
labels_to_evaluate = [label_name_to_value[key] for key in keys_present]

In [ ]:
fig1, ax1 = labelme_plot_with_custom_cmap(label_mask, value_to_label_name, road_label=0)